In [2]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

/tmp/ipykernel_3040/2424380868.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


In [3]:
from langchain_community.vectorstores import Chroma

CHROMA_DB_PATH = "./chroma_db"
COLLECTION_NAME = "pdf_docs"

def get_vectorstore():
    return Chroma(
        collection_name=COLLECTION_NAME,
        embedding_function=embeddings,
        persist_directory=CHROMA_DB_PATH
    )

In [4]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.1")

/home/raf/Escritorio/CosasRafa/ClasesActuales/PLN/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_3040/1305456209.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")


In [5]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def build_rag_chain():
    vectorstore = get_vectorstore()
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

    prompt = PromptTemplate.from_template("""
Usa el siguiente contexto para responder la pregunta.
Si no hay suficiente información, responde claramente que no está en el documento.

Contexto:
{context}

Pregunta:
{question}

Respuesta:
""")

    format_docs = lambda docs: "\n\n".join(d.page_content for d in docs)

    chain = (
        RunnableParallel(
            context=retriever | format_docs,
            question=RunnablePassthrough()
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

In [6]:
def rag_chat(question):
    chain = build_rag_chain()
    return chain.invoke(question)

In [7]:
respuesta = rag_chat("¿Cuál es la tesis de este artículo?")
print(respuesta)

/tmp/ipykernel_3040/772077193.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  return Chroma(


No está explícitamente mencionada en el texto proporcionado, pero se puede inferir que la tesis del artículo es defender el procesamiento del lenguaje natural fundamentado en un modelo semántico, en lugar de utilizar un mero reconocimiento de patrones y una interpretación-acción como los sistemas de diálogo de la década de 1970.
